## Check filenames and rename if not compatibel with windows os.

In [1]:
!pip install tqdm
import os
from tqdm import tqdm


def is_valid_filename(filename):
    # Check for reserved characters
    if any(char in filename for char in '<>:"/\\|?*'):
        return False
    
    # Check for control characters
    if any(0 <= ord(char) <= 31 for char in filename):
        return False

    # Check for reserved filenames without considering extensions
    name_without_extension = filename.split('.')[0].upper()
    if name_without_extension in ['CON', 'PRN', 'AUX', 'NUL'] + [f"COM{i}" for i in range(1, 10)] + [f"LPT{i}" for i in range(1, 10)]:
        return False
    
    # Check for filenames ending with a period or space
    if filename[-1] in ['.', ' ']:
        return False
    
    return True

def rename_file(filepath):
    base_dir, old_name = os.path.split(filepath)
    base_name, ext = os.path.splitext(old_name)
    new_name = base_name + "_renamed" + ext
    new_path = os.path.join(base_dir, new_name)
    
    counter = 1
    while not is_valid_filename(new_name) or os.path.exists(new_path):
        new_name = base_name + f"_renamed_{counter}" + ext
        new_path = os.path.join(base_dir, new_name)
        counter += 1
    
    os.rename(filepath, new_path)
    print(f"Renamed '{old_name}' to '{new_name}'")

def main():
    script_dir = os.getcwd()
    all_files = [os.path.join(dirpath, filename) 
                 for dirpath, _, filenames in os.walk(script_dir) 
                 for filename in filenames]

    changed_count = 0

    # Process each file with a progress bar from tqdm
    for filepath in tqdm(all_files, desc="Processing files", unit="file"):
        filename = os.path.basename(filepath)
        if not is_valid_filename(filename):
            rename_file(filepath)
            changed_count += 1

    print(f"\nTotal files scanned: {len(all_files)}")
    print(f"Total filenames changed: {changed_count}")

if __name__ == '__main__':
    main()



Processing files: 100%|██████████| 2046/2046 [00:00<00:00, 106661.36file/s]


Total files scanned: 2046
Total filenames changed: 0
